In [1]:
import pandas as pd

In [3]:
url = "https://en.wikipedia.org/wiki/UFC_Fight_Night:_Dern_vs._Hill"
tables = pd.read_html(url)

In [7]:
input_df = pd.read_csv(r"C:\Development\ultimateNakMuay\data\raw\wiki_events_ufc.csv")

In [8]:
input_df.head()

,#,Event,Date,Venue,Location,Attendance,Ref.
0,648,UFC Fight Night: Dern vs. Hill,"May 20, 2023",UFC Apex,"Las Vegas, Nevada, U.S.",—,[23]
1,647,UFC on ABC: Rozenstruik vs. Almeida,"May 13, 2023",Spectrum Center,"Charlotte, North Carolina, U.S.","18,712",[24]
2,646,UFC 288: Sterling vs. Cejudo,"May 6, 2023",Prudential Center,"Newark, New Jersey, U.S.","17,559",[25]
3,645,UFC on ESPN: Song vs. Simón,"Apr 29, 2023",UFC Apex,"Las Vegas, Nevada, U.S.",—,[26]
4,644,UFC Fight Night: Pavlovich vs. Blaydes,"Apr 22, 2023",[27],NaN,NaN,NaN
